<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Постановка-задачи" data-toc-modified-id="Постановка-задачи-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Постановка задачи</a></span></li><li><span><a href="#Загрузка,-обзор,-предобработка-данных" data-toc-modified-id="Загрузка,-обзор,-предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка, обзор, предобработка данных</a></span></li><li><span><a href="#Рассмотрение-метода-умножения-матриц" data-toc-modified-id="Рассмотрение-метода-умножения-матриц-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Рассмотрение метода умножения матриц</a></span></li><li><span><a href="#Выбор-и-обоснование-алгоритма-преобразования-данных" data-toc-modified-id="Выбор-и-обоснование-алгоритма-преобразования-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выбор и обоснование алгоритма преобразования данных</a></span></li><li><span><a href="#Проверка-выбранного-алгоритма-на-метриках-качества-работы-модели-машинного-обучения" data-toc-modified-id="Проверка-выбранного-алгоритма-на-метриках-качества-работы-модели-машинного-обучения-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка выбранного алгоритма на метриках качества работы модели машинного обучения</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# ПРОЕКТ
# Разработка метода преобразования данных для обеспечения защиты персональных данных клиентов страховой компании "Хоть потоп"

## Постановка задачи

Необходимо разработать метод преобразования персональных данных клиентов страховой компании "Хоть потоп", таким образом, чтобы
- с одной стороны данные было сложно восстановить;
- с другой - преобразование данных не ухудшило качество работы модели машинного обучения (линейной регрессии).

**Порядок выполнения проекта.**

Проект будет выполнен в 4 шага:
- загрузка, обзор и предобработка данных;
- рассмотрение метода умножения матриц;
- выбор и  обоснование алгоритма преобразования данных;
- проверка выбранного алгоритма на метриках качества работы модели машинного обучения.

## Загрузка, обзор, предобработка данных

In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv('/datasets/insurance.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
data.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [ ]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [ ]:
data.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


In [ ]:
data.duplicated().sum()

153

In [ ]:
data.drop_duplicates(inplace=True)

Данные не содержат пропусков, типы данных соответствуют данным, дубликаты строк удалены. Корреляция между признаками отсутствует. Определенная корреляция (0,65) существует между Возрастом и целевым признаком (Страховые выплаты), что не повлияет на качество обучения модели. Статистические характеристики данных (минимум, максимум, среднее, медиана итд) позволяют сделать вывод об отсутствии явных аномалий в данных. Сам датасет не нуждается в дополнительной предобработке, за исключением преобразования наименований столбцов:

In [ ]:
data.rename(columns = {'Пол':'gender', 'Возраст':'age', 'Зарплата':'salary',
                     'Члены семьи':'family_size', 'Страховые выплаты':'payments_count'},
          inplace = True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gender          4847 non-null   int64  
 1   age             4847 non-null   float64
 2   salary          4847 non-null   float64
 3   family_size     4847 non-null   int64  
 4   payments_count  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 227.2 KB


**Вывод по разделу:**

Полные дубликаты строк удалены. Наименование столбцов приведено в стандарту.

Данные готовы для дальнейшей работы.

## Рассмотрение метода умножения матриц

В данном разделе необходимо аргументированно ответить на вопрос: изменится ли качество линейной регрессии при умножении матрицы признаков на обратимую матрицу.

Если изменится - необходимо привести примеры матриц.

Если не изменится - указать, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $X'$ — матрица признаков, умноженная на обратимую матрицу

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Формула для $$X'$$.

$$X'= X * P$$

Тогда формула обучения принимает следующий вид:
$$ ((XP)^T XP)^{-1} (XP)^T y $$

При выполнении действий, получаем:

$$ P^{-1} (X^T X)^ {-1} ((P^T)^{-1} P^T) X^T y $$

Учитывая, что P - обратимая матрица, то $$ (P^T)^{-1} * P^T = E $$

тогда $$ P^{-1} (X^T X)^ {-1} ((P^T)^{-1} P^T) X^T y  =  P^{-1} (X^T X)^ {-1} X^T y$$

и формула обучения тогда выглядит следующим образом:

$$ w' = P^{-1} * w $$

Тогда формула предстказаний линейной регрессии выглядит следующим образом:

$$ a' = X' * w' = X P P^{-1} w $$, а, учитывая, что $$ P P ^{-1} = E$$, то $$ a' = XEw = Xw = a$$

Т.е. предсказания для матрицы признаков, умноженной на обратимую матрицу равны предсказаниям по матрице призанков.
Таким образом, умноженирем матрицы признаков на обратимую матрицу никак не влияет на качество предсказаний модели линейной регрессии.

Но, учитывая, что при умножении, изменяются веса для новых признаков - модель необходимо переубучать на новых признаках.


**Вывод по разделу**

Умножение матрицы признаков на обратимую матрицу не влияет на качество работы модели линейной реграессии при условии переобучения модели на новых признаках.

## Выбор и обоснование алгоритма преобразования данных

Алгоритмом преобразования данных выберем рассмотренный выше алгоритм умножения исходной матрицы признаков на квадратную обратимую матрицу размера, равного количеству признаков.

Сформируем датасеты признаков и целевого признака:

In [ ]:
features = data.drop('payments_count', axis = 1)
target = data['payments_count']

Создадим случайную квадратную матрицу, на которую будем умножать матрицу признаков.

In [ ]:
multiplier = np.random.randint(100, size=(features.shape[1], features.shape[1]))

Проверим полученную матрицу на обратимость:

In [ ]:
multiplier_inv = np.linalg.inv(multiplier)

Так так код выполняется без ошибок - сформированная случайная матрица - обратима.

Преобразуем датасет призаков методом StandartScaler:

In [ ]:
scaler = StandardScaler()

scaler.fit(features)
features = scaler.transform(features)

Определим веса (w) по формуле из предыдущего раздела и вектор предсказаний (а):

In [ ]:
features_ordg = features.copy()

features = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)


w = np.linalg.inv(features.T @ (features)) @ (features.T) @ (target)

In [ ]:
print(features)

[[ 1.          1.0030995   1.1754362   0.97315092 -0.18517565]
 [ 1.         -0.99691008  1.76456423 -0.19011493 -0.18517565]
 [ 1.         -0.99691008 -0.23847105 -1.89490109 -1.09546611]
 ...
 [ 1.         -0.99691008 -1.29890149 -0.60126924  0.7251148 ]
 [ 1.          1.0030995  -1.06325028 -0.72160708  1.63540526]
 [ 1.          1.0030995  -0.35629665  0.07061707 -0.18517565]]


In [ ]:
a = features @ w

Повторим расчеты для матрицы признаков умноженной на обратимую матрицу (multiplier):

In [ ]:
features_multiplied = features_ordg @ multiplier

In [ ]:
features_multiplied_ordg = features_multiplied.copy()

In [ ]:
features_multiplied = np.concatenate((np.ones((features_multiplied.shape[0], 1)), features_multiplied), axis=1)

In [ ]:
print(features_multiplied)

[[   1.           96.73983323  262.32175551  168.63138486  118.41653848]
 [   1.          121.75712742   55.14508187    6.11289152  -22.03660917]
 [   1.         -149.2287952  -246.0862322  -183.55100586 -264.88280028]
 ...
 [   1.          -51.88649768 -243.43386482 -144.57217843  -51.01821528]
 [   1.           68.21311423  -14.31003345  103.49618287   91.82729848]
 [   1.          -40.04853155   66.26523835   63.10687627   17.57504654]]


In [ ]:
w_mlt = np.linalg.inv(features_multiplied.T @ (features_multiplied)) @ (features_multiplied.T) @ (target)

In [ ]:
a_mlt = features_multiplied @ w_mlt

Определим. насколько отличаются вектор предсказаний до умножения и после.
Рассчитаем разницу элементов векторов и сложим их:

In [ ]:
(a-a_mlt).sum()

4.750965788315187e-14

Суммарная разница элементов вектора предсказаний до умножения и после близка к нулю, следовательно, примененное преобразование не оказывает влияние на качество работы модели линейной регрессии.

**Вывод по разделу:**

В качестве метода защиты персональных данных предложен метод умножения матрицы признаков на обратимую матрицу.
Проведенный расчет показал, что такое умножение не влияет на качество работы модели (векторы предсказаний до умножения и после практически совпадают).

## Проверка выбранного алгоритма на метриках качества работы модели машинного обучения

Обучим модель линейной регрессии, рассчитаем предсказания и метрику качества (R2).

In [ ]:
model = LinearRegression()

model.fit(features_ordg, target)
predictions = model.predict(features_ordg)

In [ ]:
print('Метрика качества модели линейной регрессии до преобразования признаков:')
print('R2: ',round(r2(target, predictions),6))

Метрика качества модели линейной регрессии до преобразования признаков:
R2:  0.430201


Повторим расчеты для модели, обученой на признаках после преобразования (умножения на обратимую матрицу). После чего сравним значения метрики качества.

In [ ]:
print(features_multiplied_ordg)

[[  96.73983323  262.32175551  168.63138486  118.41653848]
 [ 121.75712742   55.14508187    6.11289152  -22.03660917]
 [-149.2287952  -246.0862322  -183.55100586 -264.88280028]
 ...
 [ -51.88649768 -243.43386482 -144.57217843  -51.01821528]
 [  68.21311423  -14.31003345  103.49618287   91.82729848]
 [ -40.04853155   66.26523835   63.10687627   17.57504654]]


In [ ]:
#scaler = StandardScaler()

#scaler.fit(features_multiplied)
#features_multiplied = scaler.transform(features_multiplied)


model = LinearRegression()

model.fit(features_multiplied_ordg, target)
predictions = model.predict(features_multiplied_ordg)

In [ ]:
print('Метрика качества модели линейной регрессии после преобразования признаков:')
print('R2: ',round(r2(target, predictions),6))

Метрика качества модели линейной регрессии после преобразования признаков:
R2:  0.430201


**Вывод по разделу:**

Метрика качества модели линейной регрессии, рассчитанная на модели, обученной на исходных признаках и на преобразованных не отличаются. Следовательно, примененное преобразование не влияет на качество работы модели линейной регрессии.

**Вывод по проекту:**

В соответсвии с заданием, необходимо было предложить метод защиты персональных данных клиентов страховой компании "Хоть потоп", который бы не позволял легко восстанавливать данные и, при этом, не оказывал бы влияние на качество модели машинного обучения (линейной регрессии), которая предсказывала бы количество страховых выплат.

В результате работы над проектом был исследован датасет размером 5 000 строк и 5 столбцов.

По результатам предобработки были найдены и исключены 153 строки полных дубликатов, в остальном - данные были готовы для обработки (отсутствовали пропуски, типы данных соответствовали данным, отсутствовала корреляция между признаками).

Методом, отвечающим требованиям задачи, был выбран метод умножения матрицы признаков на обратимую матрицу.
Возможность применения этого метода была подтверждена теоритически через формулы расчета весов и вектора предсказаний.

А также практически через расчет метрики качества модели линейной регрессии по исходным признакам и по признакам после соотвествующего умножения на обратимую матрицу.

ВЫВОД: метод умножения матрицы признаков на обратимую матрицу может использоваться для защиты персональных данных, при этом не влияет на качество работы модели линейной регрессии.